<a href="https://colab.research.google.com/github/YianKim/2020_EDAProject/blob/main/uncertainty_%EA%B8%B0%EB%B0%98_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글 드라이브


In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 라이브러리 불러오기


In [55]:
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import PIL
import pickle
import random
from tqdm import tqdm
from collections import Counter

from keras.layers.core import Lambda
from keras import backend as K

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import LSTM
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import *
from keras.models import load_model

from tqdm import tqdm

## 데이터 불러오기


In [56]:
fashion_mnist = keras.datasets.fashion_mnist
(data, labels), (test_data, test_labels) = fashion_mnist.load_data()

In [57]:
train_data = data[range(10000)].reshape([10000,28,28,1])
train_labels = labels[range(10000)].reshape([10000,1])

In [58]:
unlab_data = data[range(10000,60000)].reshape([50000,28,28,1])
unlab_labels = labels[range(10000,60000)].reshape([50000,1])

In [59]:
test_data = test_data.reshape([10000,28,28,1])
test_labels = test_labels.reshape([10000,1])

In [60]:
train_labels2 = []
unlab_labels2 = []
test_labels2 = []

In [61]:
for i in range(10000):
  white = [0,0,0,0,0,0,0,0,0,0]
  white[train_labels[i][0]] = 1
  train_labels2.append(white)

for i in range(50000):
  white = [0,0,0,0,0,0,0,0,0,0]
  white[unlab_labels[i][0]] = 1
  unlab_labels2.append(white)

for i in range(10000):
  white = [0,0,0,0,0,0,0,0,0,0]
  white[test_labels[i][0]] = 1
  test_labels2.append(white)

In [62]:
train_labels = np.array(train_labels2)
unlab_labels = np.array(unlab_labels2)
test_labels = np.array(test_labels2)

## CNN

In [63]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

In [ ]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))
model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

model.fit(
          train_data,
          train_labels,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

## 모델 결과

In [65]:
def model_eval():
  pred_mu = model.predict(test_data)
  for i in range(1, 30):
    pred_mu += model.predict(test_data)
  pred_mu = pred_mu/30
  predicted_test_labels = np.argmax(pred_mu, axis=1)
  return(accuracy_score(np.argmax(test_labels, axis=1), predicted_test_labels))

In [66]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')
model_eval()

0.8858

# 라벨링

## 라벨링; 1

In [67]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

In [68]:
# Vars : 분산들
# Outs : 결과 값들

Vars = []
Outs = []

temp1 = []

for i in tqdm(range(30)):
  temp1.append(model.predict(unlab_data))

for j in tqdm(range(unlab_data.shape[0])):

  temp2 = np.array([0,0,0,0,0,0,0,0,0,0], 'float32')
  
  for i in range(30):
    temp2 += temp1[i][j]
  Outs.append(temp2/30)
  
  outputs=[]

  for i in range(30):
    outputs.append(temp1[i][j][np.argmax(temp2)])
  Vars.append(np.var(outputs))

100%|██████████| 50000/50000 [00:09<00:00, 5059.61it/s]


In [69]:
Outs = np.array(Outs)

class마다 균등하게 선택

In [70]:
Counter(np.argmax(unlab_labels, axis=1))

Counter({0: 5058,
         1: 4973,
         2: 4984,
         3: 4981,
         4: 5026,
         5: 5011,
         6: 4979,
         7: 4978,
         8: 5010,
         9: 5000})

### 불확정성 컷 1

In [ ]:
# upper bound of variance?

UB = []

for h in range(10):
  classvars = []
  for i in tqdm(range(50000)):
    if np.argmax(unlab_labels, axis=1).tolist()[i]==h:
      classvars.append(Vars[i])
  UB.append(np.percentile(classvars, 50))

100%|██████████| 50000/50000 [01:24<00:00, 589.32it/s]


In [ ]:
lowvars = []
ind = 0 

for i in Vars:
  if i <= UB[np.argmax(unlab_labels, axis=1)[ind]]:
    lowvars.append(ind)
  ind += 1

In [ ]:
highvars = []
for i in range(unlab_data.shape[0]):
  if i not in lowvars:
    highvars.append(i)

In [ ]:
# 같은 길이의 랜덤하게 고른 data들에 모델로 label 부여 후 정확도 측정
randomindx = random.sample(range(50000), 50000)
randomindx2 = randomindx[0:25000]
randomindx = randomindx[25000:50000]
accuracy_score(np.argmax(np.array(Outs)[randomindx], axis=1), np.argmax(unlab_labels[randomindx], axis=1))

0.89192

In [ ]:
# 저분산의 data들만 모아서 모델로 label 부여 후 정확도 측정 : 95% 이상
accuracy_score(np.argmax(np.array(Outs)[lowvars], axis=1), np.argmax(unlab_labels[lowvars], axis=1))

0.9728878433874247

In [ ]:
train_data_1 = np.concatenate([train_data, unlab_data[lowvars]], axis=0)
train_labels_1 = np.concatenate([train_labels, Outs[lowvars]], axis=0)

In [ ]:
shufindx = random.sample(range(train_data_1.shape[0]),train_data_1.shape[0])
train_data_1 = train_data_1[shufindx]
train_labels_1 = train_labels_1[shufindx]

In [ ]:
train_data_2 = np.concatenate([train_data, unlab_data[randomindx]], axis=0)
train_labels_2 = np.concatenate([train_labels, Outs[randomindx]], axis=0)

In [ ]:
shufindx = random.sample(range(train_data_2.shape[0]),train_data_2.shape[0])
train_data_2 = train_data_2[shufindx]
train_labels_2 = train_labels_2[shufindx]

In [ ]:
unlab_data_1 = unlab_data[highvars]
unlab_labels_1 = unlab_labels[highvars]
unlab_data_2 = unlab_data[randomindx2]
unlab_labels_2 = unlab_labels[randomindx2]

In [ ]:
Counter(np.argmax(train_labels_1, axis=1))

Counter({0: 3600,
         1: 3599,
         2: 3613,
         3: 3638,
         4: 3469,
         5: 3491,
         6: 3151,
         7: 3507,
         8: 3505,
         9: 3508})

In [ ]:
Counter(np.argmax(train_labels_2, axis=1))

Counter({0: 3409,
         1: 3473,
         2: 3582,
         3: 3546,
         4: 3512,
         5: 3488,
         6: 3448,
         7: 3581,
         8: 3447,
         9: 3514})

In [ ]:
Counter(np.argmax(unlab_labels_1, axis=1))

Counter({0: 2529,
         1: 2407,
         2: 2492,
         3: 2490,
         4: 2513,
         5: 2505,
         6: 2489,
         7: 2489,
         8: 2505,
         9: 2500})

In [ ]:
Counter(np.argmax(unlab_labels_2, axis=1))

Counter({0: 2565,
         1: 2496,
         2: 2452,
         3: 2497,
         4: 2506,
         5: 2458,
         6: 2504,
         7: 2474,
         8: 2554,
         9: 2494})

### 불확정성 컷 2

In [71]:
# upper bound of variance?

UB25 = []
UB50 = []
UB75 = []


for h in range(10):
  classvars = []
  for i in tqdm(range(50000)):
    if np.argmax(unlab_labels, axis=1).tolist()[i]==h:
      classvars.append(Vars[i])
  UB25.append(np.percentile(classvars, 25))
  UB50.append(np.percentile(classvars, 50))
  UB75.append(np.percentile(classvars, 75))

100%|██████████| 50000/50000 [01:30<00:00, 554.15it/s]


In [72]:
# UB25 < UB50 < UB75

In [73]:
vars25 = []
vars50 = []
vars75 = []
vars100 = []

ind = 0 


for i in Vars:
  if i <= UB25[np.argmax(unlab_labels, axis=1)[ind]]:
    vars25.append(ind)
  elif i <= UB50[np.argmax(unlab_labels, axis=1)[ind]]:
    vars50.append(ind)
  elif i <= UB75[np.argmax(unlab_labels, axis=1)[ind]]:
    vars75.append(ind)
  else:
    vars100.append(ind)
  ind += 1

In [74]:
k1 = random.sample(range(len(vars25)), len(vars25))
k2 = random.sample(range(len(vars50)), len(vars50))
k3 = random.sample(range(len(vars75)), len(vars75))
k4 = random.sample(range(len(vars100)), len(vars100))

lowvars = k1[0:np.int(len(k1)/2)] + k2[0:np.int(len(k2)/2)] + k3[0:np.int(len(k3)/2)] + k4[0:np.int(len(k4)/2)]
highvars = k1[np.int(len(k1)/2):len(k1)] + k2[np.int(len(k2)/2):len(k2)] + k3[np.int(len(k3)/2):len(k3)] + k4[np.int(len(k4)/2):len(k4)]

In [75]:
# 같은 길이의 랜덤하게 고른 data들에 모델로 label 부여 후 정확도 측정
randomindx = random.sample(range(50000), 50000)
randomindx2 = randomindx[0:25000]
randomindx = randomindx[25000:50000]
accuracy_score(np.argmax(np.array(Outs)[randomindx], axis=1), np.argmax(unlab_labels[randomindx], axis=1))

0.89368

In [76]:
# 층화추출
accuracy_score(np.argmax(np.array(Outs)[lowvars], axis=1), np.argmax(unlab_labels[lowvars], axis=1))

0.8886355454218169

In [77]:
train_data_1 = np.concatenate([train_data, unlab_data[lowvars]], axis=0)
train_labels_1 = np.concatenate([train_labels, Outs[lowvars]], axis=0)

In [78]:
shufindx = random.sample(range(train_data_1.shape[0]),train_data_1.shape[0])
train_data_1 = train_data_1[shufindx]
train_labels_1 = train_labels_1[shufindx]

In [79]:
train_data_2 = np.concatenate([train_data, unlab_data[randomindx]], axis=0)
train_labels_2 = np.concatenate([train_labels, Outs[randomindx]], axis=0)

In [80]:
shufindx = random.sample(range(train_data_2.shape[0]),train_data_2.shape[0])
train_data_2 = train_data_2[shufindx]
train_labels_2 = train_labels_2[shufindx]

In [81]:
unlab_data_1 = unlab_data[highvars]
unlab_labels_1 = unlab_labels[highvars]
unlab_data_2 = unlab_data[randomindx2]
unlab_labels_2 = unlab_labels[randomindx2]

In [82]:
Counter(np.argmax(train_labels_1, axis=1))

Counter({0: 3508,
         1: 3486,
         2: 3582,
         3: 3547,
         4: 3447,
         5: 3504,
         6: 3439,
         7: 3561,
         8: 3389,
         9: 3536})

In [83]:
Counter(np.argmax(train_labels_2, axis=1))

Counter({0: 3429,
         1: 3481,
         2: 3664,
         3: 3562,
         4: 3454,
         5: 3453,
         6: 3428,
         7: 3563,
         8: 3505,
         9: 3461})

In [84]:
Counter(np.argmax(unlab_labels_1, axis=1))

Counter({0: 2471,
         1: 2497,
         2: 2417,
         3: 2519,
         4: 2512,
         5: 2544,
         6: 2636,
         7: 2460,
         8: 2434,
         9: 2511})

In [85]:
Counter(np.argmax(unlab_labels_2, axis=1))

Counter({0: 2544,
         1: 2480,
         2: 2405,
         3: 2509,
         4: 2543,
         5: 2511,
         6: 2495,
         7: 2481,
         8: 2493,
         9: 2539})

## 모델링

간단한 모델에서 받은 지식를 복잡한 모델에서 학습 

uncertainty aware data vs random sampling data

In [86]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

In [87]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)


model.fit(
          train_data_1,
          train_labels_1,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/2000
383/383 [==============================] - 9s 22ms/step - loss: 1.3518 - accuracy: 0.5796 - val_loss: 0.7088 - val_accuracy: 0.7650 - lr: 0.0010
Epoch 2/2000
383/383 [==============================] - 8s 22ms/step - loss: 0.6536 - accuracy: 0.7771 - val_loss: 0.5978 - val_accuracy: 0.7984 - lr: 0.0010
Epoch 3/2000
383/383 [==============================] - 8s 22ms/step - loss: 0.5668 - accuracy: 0.8099 - val_loss: 0.5404 - val_accuracy: 0.8269 - lr: 0.0010
Epoch 4/2000
383/383 [==============================] - 9s 22ms/step - loss: 0.5271 - accuracy: 0.8246 - val_loss: 0.5126 - val_accuracy: 0.8374 - lr: 0.0010
Epoch 5/2000
383/383 [==============================] - 8s 22ms/step - loss: 0.4989 - accuracy: 0.8376 - val_loss: 0.4965 - val_accuracy: 0.8393 - lr: 0.0010
Epoch 6/2000
383/383 [==============================] - 8s 22ms/step - loss: 0.4687 - accuracy: 0.8521 - val_loss: 0.4725 - val_accuracy: 0.8545 - lr: 0.0010
Epoch 7/2000
383/383 [==============================

In [88]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn2.h5')

In [89]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn2.h5')
model_eval()

0.891

In [90]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################

model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

model.fit(
          train_data_2,
          train_labels_2,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/2000
383/383 [==============================] - 9s 22ms/step - loss: 1.1803 - accuracy: 0.6096 - val_loss: 0.6667 - val_accuracy: 0.7740 - lr: 0.0010
Epoch 2/2000
383/383 [==============================] - 8s 22ms/step - loss: 0.6082 - accuracy: 0.7905 - val_loss: 0.5672 - val_accuracy: 0.8093 - lr: 0.0010
Epoch 3/2000
383/383 [==============================] - 8s 21ms/step - loss: 0.5290 - accuracy: 0.8217 - val_loss: 0.5196 - val_accuracy: 0.8318 - lr: 0.0010
Epoch 4/2000
383/383 [==============================] - 8s 22ms/step - loss: 0.4852 - accuracy: 0.8405 - val_loss: 0.4826 - val_accuracy: 0.8488 - lr: 0.0010
Epoch 5/2000
383/383 [==============================] - 8s 22ms/step - loss: 0.4569 - accuracy: 0.8543 - val_loss: 0.4636 - val_accuracy: 0.8581 - lr: 0.0010
Epoch 6/2000
383/383 [==============================] - 9s 22ms/step - loss: 0.4396 - accuracy: 0.8632 - val_loss: 0.4338 - val_accuracy: 0.8680 - lr: 0.0010
Epoch 7/2000
383/383 [==============================

In [91]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no.h5')

In [92]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no.h5')
model_eval()

0.893

# 남은 데이터 라벨링하기


In [93]:
model1 = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn2.h5')
model2 = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no.h5')

In [94]:
unlab_label_1 = model1.predict(unlab_data_1)

In [95]:
unlab_label_2 = model2.predict(unlab_data_2)

In [96]:
train_data_1 = np.concatenate([train_data_1, unlab_data_1], axis=0)
train_labels_1 = np.concatenate([train_labels_1, unlab_label_1], axis=0)

In [97]:
train_data_2 = np.concatenate([train_data_2, unlab_data_2], axis=0)
train_labels_2 = np.concatenate([train_labels_2, unlab_label_2], axis=0)

In [ ]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)


model.fit(
          train_data_1,
          train_labels_1,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/2000
657/657 [==============================] - 17s 24ms/step - loss: 1.0846 - accuracy: 0.6199 - val_loss: 0.6432 - val_accuracy: 0.7925 - lr: 0.0010
Epoch 2/2000
657/657 [==============================] - 15s 22ms/step - loss: 0.5814 - accuracy: 0.8075 - val_loss: 0.5579 - val_accuracy: 0.8241 - lr: 0.0010
Epoch 3/2000
657/657 [==============================] - 15s 23ms/step - loss: 0.5200 - accuracy: 0.8325 - val_loss: 0.4998 - val_accuracy: 0.8548 - lr: 0.0010
Epoch 4/2000
657/657 [==============================] - 15s 23ms/step - loss: 0.4643 - accuracy: 0.8607 - val_loss: 0.4565 - val_accuracy: 0.8761 - lr: 0.0010
Epoch 5/2000
657/657 [==============================] - 15s 23ms/step - loss: 0.4392 - accuracy: 0.8724 - val_loss: 0.4518 - val_accuracy: 0.8794 - lr: 0.0010
Epoch 6/2000
657/657 [==============================] - 15s 23ms/step - loss: 0.4224 - accuracy: 0.8797 - val_loss: 0.4349 - val_accuracy: 0.8901 - lr: 0.0010
Epoch 7/2000
657/657 [========================

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn3.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn3.h5')
model_eval()

In [ ]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)


model.fit(
          train_data_2,
          train_labels_2,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no2.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no2.h5')
model_eval()